In [16]:
import urllib.request
import json
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import requests

In [100]:
current_date = datetime.now(ZoneInfo('UTC')) + timedelta(hours=-24)
current_date = current_date.replace(tzinfo=None)
next_date = datetime.now(ZoneInfo('UTC')) + timedelta(hours=24)
next_date = next_date.replace(tzinfo=None)
start_date = current_date.strftime('%Y-%m-%d')
end_date = next_date.strftime('%Y-%m-%d')

In [94]:
current_date

datetime.datetime(2025, 5, 23, 23, 21, 2, 643611)

In [171]:
# Define start_date and end_date as datetime objects
#current_date = datetime.now()
#next_date = current_date + timedelta(hours=-49)
#start_date = next_date.strftime('%Y-%m-%d')
#end_date = current_date.strftime('%Y-%m-%d')

current_date = datetime.now(ZoneInfo('UTC')) + timedelta(hours=-1)
current_date = current_date.replace(tzinfo=None)
next_date = current_date + timedelta(hours=-48)
start_date = next_date.strftime('%Y-%m-%d')
end_date = current_date.strftime('%Y-%m-%d')

print(start_date, end_date)

try:
    # Construct the URL with the provided dates
    url = f"https://apimgw.aeso.ca/public/poolprice-api/v1.1/price/poolPrice?startDate={start_date}&endDate={end_date}"

    hdr = {
        # Request headers
        'Cache-Control': 'no-cache',
        'API-KEY': 'ff8bd8112f6942ac97bb08cc88d0deae',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    print(response.getcode())

    # Read and decode the response
    response_data = response.read().decode('utf-8')
    
    # Parse the response data as JSON
    json_data = json.loads(response_data)
    
    df = pd.DataFrame(json_data)

    # Initialize an empty DataFrame to store the output
    out = pd.DataFrame()

    # Flatten the JSON data using the 'return' column
    flattened_data = pd.json_normalize(df['return']['Pool Price Report'])
    out['begin_datetime_utc'] = flattened_data['begin_datetime_utc']
    out['forecast_pool_price'] = flattened_data['pool_price']
    out['rolling_30day_avg_price'] = flattened_data['rolling_30day_avg']

    out = out[(pd.to_datetime(out.begin_datetime_utc) <= current_date) & (pd.to_datetime(out.begin_datetime_utc) >= next_date) ]
    out['begin_datetime_utc'] = pd.to_datetime(out['begin_datetime_utc'])

    out['begin_datetime_utc'] = out['begin_datetime_utc'] + timedelta(days=1)
    #out.to_csv(f'Jobs/Inferencing/data/raw/price_{start_date.replace("-", "")}_{end_date.replace("-", "")}.csv')

except Exception as e:
    print(e)

2025-05-22 2025-05-24
200


In [172]:
datetime.now()

datetime.datetime(2025, 5, 24, 18, 35, 41, 400337)

In [173]:
out

,begin_datetime_utc,forecast_pool_price,rolling_30day_avg_price
18,2025-05-24 00:00:00,37.45,35.58
19,2025-05-24 01:00:00,47.90,35.53
20,2025-05-24 02:00:00,60.15,35.43
21,2025-05-24 03:00:00,59.32,35.42
22,2025-05-24 04:00:00,33.87,35.41
23,2025-05-24 05:00:00,21.62,35.41
24,2025-05-24 06:00:00,24.53,35.41
25,2025-05-24 07:00:00,26.19,35.41
26,2025-05-24 08:00:00,24.14,35.41
27,2025-05-24 09:00:00,22.31,35.40


In [101]:
try:
    # Construct the URL with the provided dates
    url = f"https://apimgw.aeso.ca/public/actualforecast-api/v1/load/albertaInternalLoad?startDate={start_date}&endDate={end_date}"

    hdr = {
        # Request headers
        'Cache-Control': 'no-cache',
        'API-KEY': 'ff8bd8112f6942ac97bb08cc88d0deae',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    print(response.getcode())

    response_data = response.read().decode('utf-8')

    # Parse the response data as JSON
    json_data = json.loads(response_data)
    
    df = pd.DataFrame(json_data)

    # Initialize an empty DataFrame to store the output
    out = pd.DataFrame()
    
    # Flatten the JSON data using the 'return' column
    flattened_data = pd.json_normalize(df['return']['Actual Forecast Report'])
    out['begin_datetime_utc'] = flattened_data['begin_datetime_utc'] 
    out['alberta_internal_load'] = flattened_data['forecast_alberta_internal_load']
    # print(out)
    out = out[(pd.to_datetime(out.begin_datetime_utc) >= current_date) & (pd.to_datetime(out.begin_datetime_utc) <= next_date) ]

    #out.to_csv(f'Jobs/Inferencing/data/raw/AIL_{start_date.replace("-", "")}_{end_date.replace("-", "")}.csv')

except Exception as e:
    print(e)

200


In [ ]:
out

In [136]:
def get_weather_forecast(latitude, longitude):
    """
    Fetches the next 24-hour temperature and wind speed forecast for a given location.
    #Calgary --> latitude=51.0447, longitude=-114.0719
    #Edmonton --> latitude=53.5501, longitude=-113.4687
    #FortMcMurray --> latitude=56.7268, longitude=-111.381
    
    :param latitude: Latitude of the location (default: Calgary, Canada)
    :param longitude: Longitude of the location (default: Calgary, Canada)
    :return: Pandas DataFrame with columns: ['Timestamp', 'Temperature (°C)', 'Wind Speed (km/h)']
    """
    # Define the API endpoint and parameters
    endpoint = 'https://api.open-meteo.com/v1/forecast'
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': ['temperature_2m', 'windspeed_10m'],
        'timezone': 'America/Edmonton'
    }

    # Make the API request
    response = requests.get(endpoint, params=params)
    
    # Check for successful response
    if response.status_code != 200:
        print("Error fetching data:", response.text)
        return None
    
    data = response.json()

    # Extract forecast data
    hourly_times = data['hourly']['time']  # Next 48 hours
    hourly_temps = data['hourly']['temperature_2m']
    hourly_windspeeds = data['hourly']['windspeed_10m']

    # Convert time to datetime format
    timestamps = [datetime.fromisoformat(time) for time in hourly_times]

    # Create a Pandas DataFrame
    forecast_df = pd.DataFrame({
        'Timestamp': timestamps,
        'Temperature (°C)': hourly_temps,
        'Wind Speed (km/h)': hourly_windspeeds
    })

    current_date = datetime.now()
    #current_date = datetime.now(ZoneInfo("UTC"))
    #current_date = current_date.replace(tzinfo=None)
    forecast_df = forecast_df[(pd.to_datetime(forecast_df.Timestamp) > current_date) & (pd.to_datetime(forecast_df.Timestamp) < current_date+ timedelta(hours=24)) ]
    return forecast_df


df = forecast_df_calgary = get_weather_forecast(latitude=51.0447, longitude=-114.0719)
start_date = df['Timestamp'].min()
end_date = df['Timestamp'].max()


#Convert the 'Timestamp' column to datetime format and set timezone to UTC
df['Timestamp'] = df['Timestamp'].dt.tz_localize('America/Edmonton').dt.tz_convert('UTC')
df['Timestamp'] = df['Timestamp'].dt.strftime('%Y-%m-%d %H:00')

# print('completed..')
#df[['Timestamp', 'Temperature (°C)']].to_csv(f'Jobs/Inferencing/data/raw/temperature_calgary_{str(start_date).split(" ")[0].replace("-", "")}_{str(end_date).split(" ")[0].replace("-", "")}.csv')
#
#df[['Timestamp', 'Wind Speed (km/h)']].to_csv(f'Jobs/Inferencing/data/raw/windspeed_calgary_{str(start_date).split(" ")[0].replace("-", "")}_{str(end_date).split(" ")[0].replace("-", "")}.csv')

In [137]:
df[['Timestamp', 'Temperature (°C)']]

,Timestamp,Temperature (°C)
19,2025-05-25 01:00,19.5
20,2025-05-25 02:00,18.8
21,2025-05-25 03:00,16.7
22,2025-05-25 04:00,14.6
23,2025-05-25 05:00,13.1
24,2025-05-25 06:00,11.5
25,2025-05-25 07:00,10.3
26,2025-05-25 08:00,9.3
27,2025-05-25 09:00,8.5
28,2025-05-25 10:00,7.9


In [109]:

import time

url = 'http://ets.aeso.ca/Market/Reports/Manual/Operations/prodweb_reports/wind_solar_forecast/solar_rpt_longterm.csv'
# Get the current date and add one day to it
current_date = datetime.now()
next_date = current_date + timedelta(days=1)

result_df = pd.DataFrame()

while result_df.empty:
    try:
        df = pd.read_csv(url)
        df = df[(pd.to_datetime(df['Forecast Transaction Date']) > current_date) & (pd.to_datetime(df['Forecast Transaction Date']) < next_date) ]
        result_df['datetime_'] = df['Forecast Transaction Date']
        result_df['solar_generation'] = df['Most Likely']
        result_df.reset_index(drop=True, inplace=True)
        # print(result_df)
    except Exception as e:
        print(f"Error fetching data: {e}. Retrying in 5 seconds...")
        time.sleep(5)

    break

current_date = current_date.strftime('%Y-%m-%d')
next_date = next_date.strftime('%Y-%m-%d')
#result_df.to_csv(f'Jobs/Inferencing/data/raw/solar_generation_{str(current_date).replace("-", "")}_{str(next_date).replace("-", "")}.csv')

In [111]:
result_df['datetime_'] = pd.to_datetime(result_df['datetime_'], format='%Y-%m-%d %H:00')
result_df['datetime_'] = result_df['datetime_'].dt.tz_localize('America/Edmonton').dt.tz_convert('UTC')
result_df['datetime_'] = result_df['datetime_'].dt.strftime('%Y-%m-%d %H:00')

In [113]:
result_df

,datetime_,solar_generation
0,2025-05-25 00:00,1047.67
1,2025-05-25 01:00,542.07
2,2025-05-25 02:00,161.46
3,2025-05-25 03:00,6.65
4,2025-05-25 04:00,0.00
5,2025-05-25 05:00,0.00
6,2025-05-25 06:00,0.00
7,2025-05-25 07:00,0.00
8,2025-05-25 08:00,0.00
9,2025-05-25 09:00,0.00
